In [1]:
import glob
import pandas as pd
racoonf = glob.glob("racoon_train.*.jpg")
racoonans = [0] * len(racoonf)
df = pd.DataFrame({
    "path":racoonf,
    "ans":racoonans
})
df

,path,ans


In [2]:
from keras.applications.vgg16 import VGG16
vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
vgg.summary()

Using TensorFlow backend.
C:\Python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Python36\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0    

In [3]:
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
from keras.layers import BatchNormalization
for l in vgg.layers:
    #如果差很多,建議打開->True
    l.trainable = False
x = BatchNormalization()(vgg.output)
x = Flatten()(x)
x = Dense(4096, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(2, activation="softmax")(x)
cnn = Model(inputs=vgg.input, outputs=x)
cnn.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [7]:
# 進到模型處理: numpy array
import numpy as np
x = np.array(df["path"])
y = np.array(pd.get_dummies(df["ans"]))

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,
                                                    y, 
                                                    test_size=0.1)

ValueError: With n_samples=0, test_size=0.1 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [10]:
# only Demo: 藉由一張圖片告訴你如何處理
from keras.preprocessing.image import load_img
# 一定使用他當初的做法
# https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py
from keras.applications.vgg16 import preprocess_input
img = load_img(x_train[0], target_size=(224, 224)).convert("RGB")
img_np = np.array(img) 
preprocess_input(img_np)

NameError: name 'x_train' is not defined

In [ ]:
# return (原圖, 預處理過圖, 答案)
def get_images(batch, x, y):
    idx = np.random.randint(0, x.shape[0], batch)
    ori_img_list = []
    img_list = []
    for fp in x[idx]:
        img = load_img(fp, target_size=(224, 224)).convert("RGB")
        img_np = np.array(img) 
        img_pre = preprocess_input(img_np)
        img_list.append(img_pre)
        ori_img_list.append(img_np)
    img_list_np = np.array(img_list)
    ori_img_list_np = np.array(ori_img_list)
    return (ori_img_list_np, img_list_np, y[idx])

In [ ]:
cnn.compile(loss="categorical_crossentropy",
            optimizer="adam",
            metrics=["accuracy"])

In [ ]:
times = 5
for i in range(times):
    imgs = get_images(20, x_train, y_train)
    result = cnn.train_on_batch(imgs[1], imgs[2])
    print("-" * 15, "times", i + 1, "-" * 15)
    print("[Train]:", result)
    # 如果時間充裕, 最好多一點
    test_imgs = get_images(50, x_test, y_test)
    test_result = cnn.test_on_batch(test_imgs[1], test_imgs[2])
    print("[Validate]:", test_result)

In [ ]:
cnn.predict(test_imgs[1])

In [ ]:
test_imgs = get_images(200, x_test, y_test)
cnn.evaluate(test_imgs[1], test_imgs[2])

In [ ]:
f = get_images(50, x_test, y_test)
pre = cnn.predict(f[1]).argmax(axis=1)
pre

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
labels = ["cat", "dog"]
test = f[2].argmax(axis=1)
idx = np.nonzero(pre != test)[0]
false_img = f[0][idx]
false_answer = test[idx]
false_pre = pre[idx]

width = 10
height = len(false_img) // width + 1
plt.figure(figsize=(14, 4))
for i, img in enumerate(false_img):
    plt.subplot(height, width, i + 1)
    t = "[O]:" + labels[false_answer[i]] + "\n[P]:" + labels[false_pre[i]]
    plt.title(t)
    plt.axis("off")
    plt.imshow(img)